In [259]:
# TODO:
# 1. Process all games chronologically 
# 2. Fix the issue of a player being missing the mapping JSON. Right now they won't be processed.
# 3. Write team rating into a JSON for storage?

from glob import glob
import json

# Constants
player_id = "player_id"
handle = "handle"

platformGameId = "platformGameId"
participantMapping = "participantMapping"
participantId = "participantId"

gameInfoEventInd = 0
platformGameId = "platformGameId"
eventType = "eventType"
champion_kill = "champion_kill"
killer = "killer"
game_info = "game_info"

STARTING_TEAM_RATING = 1000
MINIMUM_TEAM_RATING = 0 
ENTRY_RATING_FEE = 150 # 'SR' cost to play a game
WIN_MATCH_SR_GAIN = 100
KILL_SR_GAIN = 10
WARD_KILL_SR_GAIN = 1
TOWER_KILL_SR_GAIN = 15
DRAGON_KILL_SR_GAIN = 10
RIFT_KILL_SR_GAIN = 10
BARON_KILL_SR_GAIN = 20

# Global values
playerIDPlayerMap = {}
teamIDTeamMap = {}
mappingDataMap = {}
matchIdTeams = {}

# Helper Functions
# Returns a storage based on read contents of file
def readJson(fileName):
    with open(fileName) as file:
        file_contents = file.read()
    print("Processing file: " + fileName)
    return json.loads(file_contents)

# Helper Objects
# Player object to represent a pro player
class Player():
    def __init__(self, playerInfo):
        # Identification info
        self._playerId = playerInfo[player_id]
        self._handle = playerInfo[handle]

        # Games' specific info
        self._matchCount = 0
        self._totalKills = 0 
    
    # Return average kills per match
    def getAvgKillsPerMatch(self):
        # Note: protects against 0 matches played
        return round(self._totalKills / max(self._matchCount, 1), 2)

    def addToMatchCount(self, toAdd):
        self._matchCount += toAdd
    
    def addToTotalKill(self, toAdd):
        self._totalKills += toAdd

    def getPlayerId(self):
        return self._playerId
    
    def getHandle(self):
        return self._handle
    
    def getMatchCount(self):
        return self._matchCount

    def getTotalKills(self):
        return self._totalKills

# Team represents a pro team
class Team():
    def __init__(self, teamInfo):
        self._teamId = teamInfo['team_id']
        self._name = teamInfo['name']
        self._rating = STARTING_TEAM_RATING
        self._wins = 0
        self._losses = 0
        self._kills = 0

    def getTeamId(self):
        return self._teamId

    def getName(self):
        return self._name
    
    def getRating(self):
        return self._rating

    def addRating(self, addRating):
        self._rating += addRating
    
    def subtractRating(self, subtractRating):
        self._rating -= subtractRating
        # Rating can't go below zero
        self._rating = max(self._rating, MINIMUM_TEAM_RATING)

    def getWins(self):
        return self._wins
    
    def addWin(self):
        self._wins += 1
    
    def getLosses(self):
        return self._losses

    def addLoss(self):
        self._losses += 1

    def getKills(self):
        return self._kills

    def addKills(self, killAddition):
        self._kills += killAddition


In [260]:
# Build player metadata
# Map of { player_id : playerObjs }
# Note: Can store all players ever since count of all pro players ever should be under order of 10 thousands

# Read player.json to build this map
playerJson = readJson('esports-data/players.json')
for player in playerJson:
    # TODO: Guard against player w/o id or handle 
    playerIDPlayerMap[player[player_id]] = Player(player)

print("Evaluating across " + str(len(playerIDPlayerMap)) + " players\n")

Processing file: esports-data/players.json
Evaluating across 4219 players



In [261]:
# Build team metadata
# Map of { team_id: teamObjs }
# Read player.json to build this map
teamJson = readJson('esports-data/teams.json')
for team in teamJson:
    teamIDTeamMap[team['team_id']] = Team(team)

print("Evaluating across " + str(len(teamIDTeamMap)) + " teams\n")


Processing file: esports-data/teams.json
Evaluating across 673 teams



In [262]:
# Build the mapping data from JSON of same name
# Map of { platformGameId : { participantId : player_id }}
mappingJson = readJson('esports-data/mapping_data.json')
for mapping in mappingJson:
    # TODO: Guard against participant mapping missing
    currParticipantMapping = mapping[participantMapping]
    mappingDataMap[mapping[platformGameId]] = currParticipantMapping

Processing file: esports-data/mapping_data.json


In [263]:
# Read tournament JSON to build a map of { match_id: 100: team_id_blue, 200: team_id_red, winner: 100 OR 200 OR 0 }
# 0 means tie
tournamentJson = readJson('esports-data/tournaments.json')
for tournament in tournamentJson:
    if 'stages' in tournament:
        for stage in tournament['stages']:
            if 'sections' in stage:
                for section in stage['sections']:
                    if 'matches' in section:
                        for match in section['matches']:
                            if 'id' in match and 'teams' in match and len(match['teams']) == 2:
                                if 'games' in match:
                                    for game in match['games']:
                                        key = match['id'] + '|game' + str(game['number'])
                                        matchIdTeams[key] = {'winningSide': 0}

                                        # Side for team
                                        for team in game['teams']:
                                            if team['side'] == 'blue':
                                                matchIdTeams[key][100] = team['id']
                                            else:
                                                matchIdTeams[key][200] = team['id']
                                            if 'result' in team and team['result'] and 'outcome' in team['result'] and team['result']['outcome'] == 'win':
                                                matchIdTeams[key]['winningSide'] = 100 if team['side'] == 'blue' else 200

print("Evaluating across " + str(len(matchIdTeams)) + " matches\n")

Processing file: esports-data/tournaments.json
Evaluating across 28670 matches



In [264]:
def getWinnerLoser(winLossMap):
    if winLossMap['winningSide'] == 100:
        return winLossMap[100], winLossMap[200]
    elif winLossMap['winningSide'] == 200:
        return winLossMap[200], winLossMap[100]
    
    return None, None

# Object to hold information both team's performance for a match
class TeamsInMatch:
    def __init__(self, teamID100, teamID200):
        self._teamID100 = teamID100
        self._teamID200 = teamID200
        self._players100 = set()
        self._players200 = set()
        self._srGain100 = 0
        self._srGain200 = 0
        self._killCount100 = 0
        self._killCount200 = 0
    
    def getTeamID(self, sideID):
        if sideID == 100:
            return self._teamID100
        elif sideID == 200:
            return self._teamID200

    def addPlayer(self, sideID, playerID):
        if sideID == 100:
            self._players100.add(playerID)
        elif sideID == 200:
            self._players200.add(playerID)
    
    def addKill(self, playerID):
        if playerID in self._players100:
            self._srGain100 += KILL_SR_GAIN
            self._killCount100 += 1
        elif playerID in self._players200:
            self._srGain200 += KILL_SR_GAIN
            self._killCount200 += 1
        else:
            print(str(playerID) + " not found for match")
    
    def getTeamIDSRGain(self, teamSide):
        if teamSide == 100:
            return self._teamID100, self._srGain100
        elif teamSide == 200:
            return self._teamID200, self._srGain200
    
    def getTeamIDKillGain(self, teamSide):
        if teamSide == 100:
            return self._teamID100, self._killCount100
        elif teamSide == 200:
            return self._teamID200, self._killCount200


# Read each JSON in ./games by order of game occurance
# PER Game
# Reading participant 
# Using mapping_data.json create a map of { participantID (1 - 10 w.r.t match) : player_id } 
# TODO: Loop through all games in .games/
for f_name in glob('games/*.json'):
    currGameJson = readJson(f_name)

    # TODO: For now, drop a game if first event isn't about the game
    #  Consider altering in the future
    gameInfoEvent = currGameJson[gameInfoEventInd]
    if gameInfoEvent[eventType] != game_info:
        continue

    gameName = gameInfoEvent['gameName']
    if not gameName in matchIdTeams:
        continue

    # Winning and losing team of the match
    winner, loser = getWinnerLoser(matchIdTeams[gameName])
    if winner == None or loser == None:
        continue
    teamIDTeamMap[winner].addWin()
    teamIDTeamMap[loser].addLoss()
    print('Winner: ' + teamIDTeamMap[winner].getName() + " | Loser: " + teamIDTeamMap[loser].getName())

    # Create object to track team performance for the match
    teamInMatch = TeamsInMatch(matchIdTeams[gameName][100], matchIdTeams[gameName][200])

    # A many to two map which maps player ID to team SR objects
    incompletePlayer = False
    for participant in gameInfoEvent['participants']:
        if not 'teamID' in participant or not 'participantID' in participant:
            incompletePlayer = True
            break
        teamInMatch.addPlayer(participant['teamID'], participant['participantID'])
    if incompletePlayer:
        print('Ignoring match, incomplete match info')
        continue

    # Original SRs
    orgSR100 = teamIDTeamMap[matchIdTeams[gameName][100]].getRating()
    orgSR200 = teamIDTeamMap[matchIdTeams[gameName][200]].getRating()

    # SR calculations
    # 1. Teams 'pay' entry SR
    teamIDTeamMap[winner].subtractRating(ENTRY_RATING_FEE)
    teamIDTeamMap[winner].addRating(WIN_MATCH_SR_GAIN)
    teamIDTeamMap[loser].subtractRating(ENTRY_RATING_FEE)

    # Determine participant Id to PlayerId mapping via platformGameId
    currPartIdPlayerId = mappingDataMap[gameInfoEvent[platformGameId]]

    # Add one match played to each player
    for participantId in currPartIdPlayerId:
        playerIDPlayerMap[currPartIdPlayerId[participantId]].addToMatchCount(1)

    # Per CHAMPION_KILL event in game. +1 to kill_score for the killer player_id
    for event in currGameJson:
        if event[eventType] == champion_kill: 
            killerId = str(event[killer])
            if killerId in currPartIdPlayerId:
                playerIDPlayerMap[currPartIdPlayerId[killerId]].addToTotalKill(1)
                teamInMatch.addKill(int(killerId))
    
    # Provide SR gain and kill gain for both teams
    team100Id, team100SRGain = teamInMatch.getTeamIDSRGain(100)
    teamIDTeamMap[team100Id].addRating(team100SRGain)
    _, team100KillGain = teamInMatch.getTeamIDKillGain(100)
    teamIDTeamMap[team100Id].addKills(team100KillGain)

    team200Id, team200SRGain = teamInMatch.getTeamIDSRGain(200)
    teamIDTeamMap[team200Id].addRating(team200SRGain)
    _, team200KillGain = teamInMatch.getTeamIDKillGain(200)
    teamIDTeamMap[team200Id].addKills(team200KillGain)
    
    print(teamIDTeamMap[team100Id].getName())
    print('> SR change: ' + str(teamIDTeamMap[team100Id].getRating() - orgSR100) + ' | ' + str(orgSR100) + ' => ' + str(teamIDTeamMap[team100Id].getRating()))
    print('> Kills: ' + str(team100KillGain))
    print(teamIDTeamMap[team200Id].getName())
    print('> SR change: ' + str(teamIDTeamMap[team200Id].getRating() - orgSR200) + ' | ' + str(orgSR200) + ' =>' + str(teamIDTeamMap[team200Id].getRating()))
    print('> Kills: ' + str(team200KillGain))

    print('\n')

Processing file: games/ESPORTSTMNT03:3184143.json
Winner: Anorthosis Famagusta Esports | Loser: Gamespace Mediterranean College Esports
Anorthosis Famagusta Esports
> SR change: 650 | 2000 => 2650
> Kills: 35
Gamespace Mediterranean College Esports
> SR change: 260 | 2000 =>2260
> Kills: 23


Processing file: games/ESPORTSTMNT03:3198199.json
Winner: Komodo | Loser: Return of the Middlesticks
Komodo
> SR change: 250 | 2000 => 2250
> Kills: 15
Return of the Middlesticks
> SR change: -160 | 2000 =>1840
> Kills: 2


Processing file: games/ESPORTSTMNT01:3371651.json
Winner: Wild Panthers Esports | Loser: Pentagon Rejects
Wild Panthers Esports
> SR change: 310 | 2000 => 2310
> Kills: 18
Pentagon Rejects
> SR change: 0 | 2000 =>2000
> Kills: 10


Processing file: games/ESPORTSTMNT03:3198623.json
Winner: Winthrop University | Loser: Lit Esports
Winthrop University
> SR change: 510 | 2000 => 2510
> Kills: 28
Lit Esports
> SR change: 60 | 2000 =>2060
> Kills: 13


Processing file: games/ESPORTST

In [265]:
# Print out all the teams by order of SR. This is the ranking of teams
teamIDTeamMap = dict(sorted(teamIDTeamMap.items(), key = lambda item: item[1].getRating(), reverse=True))
itter = 0
for teamID in teamIDTeamMap:
    teamObj = teamIDTeamMap[teamID]
    print(teamObj.getName())
    print('SR:' + str(teamObj.getRating()))
    print('Kills:' + str(teamObj.getKills()))
    print('Wins:' + str(teamObj.getWins()))
    print('Lossess:' + str(teamObj.getLosses()))
    print('\n')
    itter += 1
    if itter == 250:
        break

Mirage Alliance
SR:6170
Kills:246
Wins:11
Lossess:1


Anorthosis Famagusta Esports
SR:6050
Kills:245
Wins:13
Lossess:1


Team Coachify
SR:5950
Kills:265
Wins:7
Lossess:5


Team Ambition
SR:5570
Kills:266
Wins:7
Lossess:7


Winthrop University
SR:5250
Kills:230
Wins:7
Lossess:5


WLGaming Esports
SR:5050
Kills:190
Wins:7
Lossess:2


Team E Turner
SR:4670
Kills:166
Wins:5
Lossess:2


Lit Esports
SR:4030
Kills:134
Wins:5
Lossess:2


Panathinaikos Esports
SR:3800
Kills:135
Wins:6
Lossess:3


Teamless Revenge
SR:3510
Kills:123
Wins:3
Lossess:4


Gamespace Mediterranean College Esports
SR:3450
Kills:130
Wins:3
Lossess:5


Apex Mission Impossible
SR:3420
Kills:131
Wins:4
Lossess:5


Return of the Middlesticks
SR:3300
Kills:120
Wins:2
Lossess:5


Pentagon Rejects
SR:3030
Kills:119
Wins:3
Lossess:6


Gentle Hearts Gaming
SR:2980
Kills:99
Wins:4
Lossess:4


Team Phantasma
SR:2940
Kills:137
Wins:4
Lossess:8


CB Gaming
SR:2680
Kills:79
Wins:2
Lossess:4


Kelyx's Grandpa Gamers
SR:2460
Kills:58
Wi

In [266]:
# TODO: Remove
# Prints out average kill of all players that have played at least one game
for player in playerIDPlayerMap.values():
    if player.getMatchCount() > 0:
        print(player.getHandle())
        print("Avg kill per match: " + str(player.getAvgKillsPerMatch()))
        print("Totale kills: " + str(player.getTotalKills()))
        print("Matches played: " + str(player.getMatchCount()))
        print("\n")

Ssaiko
Avg kill per match: 1.5
Totale kills: 3
Matches played: 2


Kookykrook
Avg kill per match: 3.0
Totale kills: 21
Matches played: 7


Mixtsure
Avg kill per match: 0.5
Totale kills: 3
Matches played: 6


Decoy1
Avg kill per match: 1.22
Totale kills: 11
Matches played: 9


Zev
Avg kill per match: 3.0
Totale kills: 6
Matches played: 2


ROCKBOOM
Avg kill per match: 4.43
Totale kills: 31
Matches played: 7


Kisno
Avg kill per match: 4.0
Totale kills: 28
Matches played: 7


Frost1
Avg kill per match: 3.55
Totale kills: 39
Matches played: 11


Max108
Avg kill per match: 3.57
Totale kills: 25
Matches played: 7


PCL
Avg kill per match: 2.5
Totale kills: 10
Matches played: 4


Bruhruto
Avg kill per match: 0.33
Totale kills: 1
Matches played: 3


Only35
Avg kill per match: 0.33
Totale kills: 4
Matches played: 12


Antcliff
Avg kill per match: 1.5
Totale kills: 3
Matches played: 2


Rain1
Avg kill per match: 3.6
Totale kills: 18
Matches played: 5


Artoria
Avg kill per match: 4.0
Totale kil